This note book has 5 main steps:


1. Load the checkpoint
2. Score the file created by mBART by translating `english` to `tamil`
3. Score the file created by IndicTrans by translating `english` to `tamil`
4. Combine the logits from step 2, step 3, with the original sentence and the translated senences. This is also saved for further analysis
5. Apply `softmax` on the logits from step 2 and 3, resulting in `selecting` the better translations

In [1]:
%cd indic-bert

/w266/w266/project/finetuning/indic-bert


In [2]:
import glob
import os

In [3]:
from fine_tune.modules import get_modules
from fine_tune.data import load_dataset
from fine_tune.data.examples import *
from fine_tune.modules import base

In [4]:
import pandas as pd
import numpy as np
np.set_printoptions(precision=5)

In [5]:
from transformers import (
    AdamW,
    AutoConfig,
    AutoModel,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    get_linear_schedule_with_warmup,
)

In [6]:
# Calculate softmax
def _softmax(x1, x2, tog=0):
    a = np.exp([x1,x2])/np.sum(np.exp([x1,x2]))
    if tog == 1:
        return a[0]
    else:
        return a[1]

def apply_softmax(df, tog):
    return df.apply(lambda row: _softmax(row[0], row[1], tog), axis=1)

In [7]:
# Hyperparameters for testing
hparams = {
    'dataset': 'amrita-paraphrase-exact', 
    'model': 'ai4bharat/indic-bert', 
    'config_name': '', 
    'tokenizer_name': '', 
    'max_seq_length': 256, 
    'iglue_dir': '../indic-glue', 
    'overwrite_cache': True, 
    'cache_dir': '../ib-cache', 
    'fp16': False, 
    'fp16_opt_level': 'O1', 
    'n_gpu': 1, 
    'n_tpu_cores': 0, 
    'max_grad_norm': 1.0, 
    'do_train': True, 
    'do_predict': True, 
    'gradient_accumulation_steps': 1,
    'seed': 8, 
    'learning_rate': 2e-05, 
    'weight_decay': 0.0, 
    'adam_epsilon': 1e-08, 
    'warmup_steps': 0, 
    'num_train_epochs': 1, 
    'train_batch_size': 16, 
    'eval_batch_size': 32, 
    'labels': '', 
    'model_name_or_path': 'ai4bharat/indic-bert', 
    'data_dir': '../indic-glue/amrita-paraphrase-exact', 
    'mode': 'sequence-classification', 
    'output_mode': 'regression', #'classification', 
    'example_type': 'text', 
    'num_labels': 2
}

In [8]:
# Set up parameters for malayalam
hparams['out_dir'] = '../outputs/5/'
hparams['lang'] = 'ma' 
hparams['train_lang'] = 'ma'
hparams['dev_lang'] = 'ma'
hparams['test_lang'] = 'ma'
hparams['output_dir'] = '{}amrita-paraphrase-exact/train-{}/model-ai4bharat-indic-bert'.format(hparams['out_dir'],hparams['lang'] ) 

In [9]:
config_name = hparams['config_name'] or hparams['model_name_or_path']
args = {'num_labels': hparams['num_labels']}
config = AutoConfig.from_pretrained(
    config_name,
    **args,
    cache_dir=hparams['cache_dir']
)

## Score mBART Tamil

In [10]:
! rm ../indic-glue/amrita-paraphrase-exact/cached*

In [11]:
!cp ../indic-glue/amrita-paraphrase-exact/ma/ma-bart-test.csv ../indic-glue/amrita-paraphrase-exact/ma/ma-test.csv 

In [12]:
## Check if you have the right file
mbart_test = pd.read_csv('../indic-glue/amrita-paraphrase-exact/ma/ma-test.csv', header=None)
mbart_test.shape

(1000, 3)

In [13]:
# module_name = 'text_classification'

text_classification_trainer = get_modules('text_classification')
text_classification_trainer

fine_tune.modules.text_classification.TextClassification

In [14]:
tc_trainer_instance = text_classification_trainer(hparams)
checkpoints = list(sorted(glob.glob(os.path.join(hparams['output_dir'], 'checkpointepoch=*.ckpt'), recursive=True)))
trained_model = tc_trainer_instance.load_from_checkpoint(checkpoints[-1])

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertForSequenceClassification: ['predictions.bias', 'predictions.LayerNorm.weight', 'predictions.LayerNorm.bias', 'predictions.dense.weight', 'predictions.dense.bias', 'predictions.decoder.weight', 'predictions.decoder.bias', 'sop_classifier.classifier.weight', 'sop_classifier.classifier.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/indi

In [15]:
trainer = base.create_trainer(trained_model, hparams) #BaseModule(hparams)

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: Checkpoint directory ../outputs/5/amrita-paraphrase-exact/train-ma/model-ai4bharat-indic-bert exists and is not empty with save_top_k != 0.All files in this directory will be deleted when a checkpoint is saved!
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


In [16]:
trainer.test(trained_model)

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Logits written to :../outputs/5/amrita-paraphrase-exact/train-ma/model-ai4bharat-indic-bert/test_results_ma.csv
--------------------------------------------------------------------------------
TEST RESULTS
{'acc': 0.411,
 'avg_test_loss': 1.637779712677002,
 'val_loss': 1.637779712677002}
--------------------------------------------------------------------------------
{'avg_test_loss': 1.637779712677002, 'val_loss': 1.637779712677002, 'acc': 0.411}



## Load the scores for mBART translations.

In [17]:
hparams['results_file'] = '{}{}/train-{}/model-ai4bharat-indic-bert/test_results_{}.csv'.format(hparams['out_dir'],  hparams['dataset'] , hparams['lang'] , hparams['lang']) 
hparams['results_file']

'../outputs/5/amrita-paraphrase-exact/train-ma/model-ai4bharat-indic-bert/test_results_ma.csv'

In [18]:
mbart = pd.read_csv(hparams['results_file'], header=None)

In [19]:
mbart.shape

(1000, 4)

In [20]:
mbart.head(5)

,0,1,2,3
0,-0.535944,0.373010,1.0,1.0
1,1.164668,-1.398593,0.0,1.0
2,1.247734,-1.467341,0.0,1.0
3,-0.645948,0.544526,1.0,1.0
4,-0.914935,0.929445,1.0,1.0


## Score IndicTrans Tamil

In [21]:
! rm ../indic-glue/amrita-paraphrase-exact/cached*

In [22]:
!cp ../indic-glue/amrita-paraphrase-exact/ma/ma-indictrans-test.csv ../indic-glue/amrita-paraphrase-exact/ma/ma-test.csv 


In [23]:
## Check if you have the right file
mbart_test = pd.read_csv('../indic-glue/amrita-paraphrase-exact/ma/ma-test.csv', header=None)
mbart_test.shape

(1000, 3)

In [24]:
# module_name = 'text_classification'

text_classification_trainer = get_modules('text_classification')
text_classification_trainer

fine_tune.modules.text_classification.TextClassification

In [25]:
tc_trainer_instance = text_classification_trainer(hparams)
checkpoints = list(sorted(glob.glob(os.path.join(hparams['output_dir'], 'checkpointepoch=*.ckpt'), recursive=True)))
trained_model = tc_trainer_instance.load_from_checkpoint(checkpoints[-1])

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertForSequenceClassification: ['predictions.bias', 'predictions.LayerNorm.weight', 'predictions.LayerNorm.bias', 'predictions.dense.weight', 'predictions.dense.bias', 'predictions.decoder.weight', 'predictions.decoder.bias', 'sop_classifier.classifier.weight', 'sop_classifier.classifier.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/indi

In [26]:
trainer = base.create_trainer(trained_model, hparams) #BaseModule(hparams)

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: Checkpoint directory ../outputs/5/amrita-paraphrase-exact/train-ma/model-ai4bharat-indic-bert exists and is not empty with save_top_k != 0.All files in this directory will be deleted when a checkpoint is saved!
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


In [27]:
trainer.test(trained_model)

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Logits written to :../outputs/5/amrita-paraphrase-exact/train-ma/model-ai4bharat-indic-bert/test_results_ma.csv
--------------------------------------------------------------------------------
TEST RESULTS
{'acc': 0.22,
 'avg_test_loss': 1.5316938161849976,
 'val_loss': 1.5316938161849976}
--------------------------------------------------------------------------------
{'avg_test_loss': 1.5316938161849976, 'val_loss': 1.5316938161849976, 'acc': 0.22}



## Load the scores for IndicTrans translations.

In [28]:
hparams['results_file'] = '{}{}/train-{}/model-ai4bharat-indic-bert/test_results_{}.csv'.format(hparams['out_dir'],  hparams['dataset'] , hparams['lang'] , hparams['lang']) 
hparams['results_file']

'../outputs/5/amrita-paraphrase-exact/train-ma/model-ai4bharat-indic-bert/test_results_ma.csv'

In [29]:
indictrans = pd.read_csv(hparams['results_file'], header=None)

In [30]:
indictrans.shape

(1000, 4)

In [31]:
indictrans.head(5)

,0,1,2,3
0,-1.021923,0.976919,1.0,0.0
1,0.506923,-0.750715,0.0,0.0
2,-0.971697,0.945597,1.0,0.0
3,-1.004650,0.952055,1.0,0.0
4,-0.951616,0.937276,1.0,0.0


## Combine logits, original sentence, and translations

In [32]:
indic_orig = pd.read_csv('../indic-glue/amrita-paraphrase-exact/ma/ma-indictrans-test.csv', header=None)

In [33]:
indic_orig.head()

,0,1,2
0,കോടിക്കണക്കിന് ജനങ്ങളുടെ മനസില്‍ അഭിലാഷങ്ങള്‍ ...,കോടിക്കണക്കിന് ജനങ്ങളുടെ മനസ്സിലും ഹൃദയത്തിലും...,NP
1,രാജ്യത്തിന്റെ വികസയാത്രയില്‍ പിന്നോക്കം വിട്ടു...,"""ബിജാപ്പൂരിൽ ഇന്ന് ഈ പരിപാടി സംഘടിപ്പിക്കുന്നത...",NP
2,2022 ഓടെ ഈ ലക്ഷ്യം പൂര്‍ത്തീകരിക്കുകയാണ് ലക്ഷ്...,2022 ഓടെ ഈ ദൌത്യം പൂർത്തിയാക്കുകയാണ് ലക്ഷ്യമെന...,NP
3,സമയബന്ധിതമായും കൃത്യമായ ലക്ഷ്യങ്ങളോടുംകൂടി ഗവണ...,വ്യക്തമായ ലക്ഷ്യത്തോടെയും സമയക്രമത്തോടെയുമാണ് ...,NP
4,ജന്‍ ധന്‍ അക്കൗണ്ടുകള്‍ തുറക്കുന്നതിലും ദരിദ്ര...,"""ജൻധൻ അക്കൌണ്ടുകൾ തുറക്കൽ, പാവപ്പെട്ടവരുടെ ഇൻഷ...",NP


In [34]:
bart_orig = pd.read_csv('../indic-glue/amrita-paraphrase-exact/ma/ma-bart-test.csv', header=None)

In [35]:
bart_orig.head()

,0,1,2
0,കോടിക്കണക്കിന് ജനങ്ങളുടെ മനസില്‍ അഭിലാഷങ്ങള്‍ ...,കോടിക്കണക്കിന് ജനങ്ങളുടെ മനസ്സിലും മനസ്സിലും അ...,P
1,രാജ്യത്തിന്റെ വികസയാത്രയില്‍ പിന്നോക്കം വിട്ടു...,ഇന്ന് ബിജ്യപ്പூரில் ഈ പരിപാടി നടത്തുന്നതിന് റെ...,P
2,2022 ഓടെ ഈ ലക്ഷ്യം പൂര്‍ത്തീകരിക്കുകയാണ് ലക്ഷ്...,അദ്ധേഹത്തിന് റെ ലക്ഷ്യം 2022-ല് ഈ ജോലി പൂര് ത്...,P
3,സമയബന്ധിതമായും കൃത്യമായ ലക്ഷ്യങ്ങളോടുംകൂടി ഗവണ...,കൃത്യമായ ലക്ഷ്യങ്ങളും കൃത്യസമയങ്ങളും ഉള്ള பல்வ...,P
4,ജന്‍ ധന്‍ അക്കൗണ്ടുകള്‍ തുറക്കുന്നതിലും ദരിദ്ര...,"ഈ കാര്യത്തില് അദ്ദേഹം, ജാന് ദന് അക്കൗണ്ട് തുറക...",P


In [36]:
combined = bart_orig[[0, 1]]

In [37]:
combined.columns =['Original_Sentence', 'mBART_Translated', ]

In [38]:
combined['IndicTrans_Translated'] = indic_orig[[1]]

In [39]:
combined.head()

,Original_Sentence,mBART_Translated,IndicTrans_Translated
0,കോടിക്കണക്കിന് ജനങ്ങളുടെ മനസില്‍ അഭിലാഷങ്ങള്‍ ...,കോടിക്കണക്കിന് ജനങ്ങളുടെ മനസ്സിലും മനസ്സിലും അ...,കോടിക്കണക്കിന് ജനങ്ങളുടെ മനസ്സിലും ഹൃദയത്തിലും...
1,രാജ്യത്തിന്റെ വികസയാത്രയില്‍ പിന്നോക്കം വിട്ടു...,ഇന്ന് ബിജ്യപ്പூரில் ഈ പരിപാടി നടത്തുന്നതിന് റെ...,"""ബിജാപ്പൂരിൽ ഇന്ന് ഈ പരിപാടി സംഘടിപ്പിക്കുന്നത..."
2,2022 ഓടെ ഈ ലക്ഷ്യം പൂര്‍ത്തീകരിക്കുകയാണ് ലക്ഷ്...,അദ്ധേഹത്തിന് റെ ലക്ഷ്യം 2022-ല് ഈ ജോലി പൂര് ത്...,2022 ഓടെ ഈ ദൌത്യം പൂർത്തിയാക്കുകയാണ് ലക്ഷ്യമെന...
3,സമയബന്ധിതമായും കൃത്യമായ ലക്ഷ്യങ്ങളോടുംകൂടി ഗവണ...,കൃത്യമായ ലക്ഷ്യങ്ങളും കൃത്യസമയങ്ങളും ഉള്ള பல்வ...,വ്യക്തമായ ലക്ഷ്യത്തോടെയും സമയക്രമത്തോടെയുമാണ് ...
4,ജന്‍ ധന്‍ അക്കൗണ്ടുകള്‍ തുറക്കുന്നതിലും ദരിദ്ര...,"ഈ കാര്യത്തില് അദ്ദേഹം, ജാന് ദന് അക്കൗണ്ട് തുറക...","""ജൻധൻ അക്കൌണ്ടുകൾ തുറക്കൽ, പാവപ്പെട്ടവരുടെ ഇൻഷ..."


In [41]:
combined['mBART_NP_prob'] = mbart[[0]]
combined['mBART_P_prob'] = mbart[[1]]
combined['model_pred_mbart'] = ['NP' if x==0 else 'P' for x in mbart[2]]
combined['mbart_P'] = apply_softmax(mbart, 0)
combined['mbart_NP'] = apply_softmax(mbart, 1)

In [42]:
combined.head()

,Original_Sentence,mBART_Translated,IndicTrans_Translated,mBART_NP_prob,mBART_P_prob,model_pred_mbart,mbart_P,mbart_NP
0,കോടിക്കണക്കിന് ജനങ്ങളുടെ മനസില്‍ അഭിലാഷങ്ങള്‍ ...,കോടിക്കണക്കിന് ജനങ്ങളുടെ മനസ്സിലും മനസ്സിലും അ...,കോടിക്കണക്കിന് ജനങ്ങളുടെ മനസ്സിലും ഹൃദയത്തിലും...,-0.535944,0.373010,P,0.712786,0.287214
1,രാജ്യത്തിന്റെ വികസയാത്രയില്‍ പിന്നോക്കം വിട്ടു...,ഇന്ന് ബിജ്യപ്പூரில் ഈ പരിപാടി നടത്തുന്നതിന് റെ...,"""ബിജാപ്പൂരിൽ ഇന്ന് ഈ പരിപാടി സംഘടിപ്പിക്കുന്നത...",1.164668,-1.398593,NP,0.071541,0.928459
2,2022 ഓടെ ഈ ലക്ഷ്യം പൂര്‍ത്തീകരിക്കുകയാണ് ലക്ഷ്...,അദ്ധേഹത്തിന് റെ ലക്ഷ്യം 2022-ല് ഈ ജോലി പൂര് ത്...,2022 ഓടെ ഈ ദൌത്യം പൂർത്തിയാക്കുകയാണ് ലക്ഷ്യമെന...,1.247734,-1.467341,NP,0.062090,0.937910
3,സമയബന്ധിതമായും കൃത്യമായ ലക്ഷ്യങ്ങളോടുംകൂടി ഗവണ...,കൃത്യമായ ലക്ഷ്യങ്ങളും കൃത്യസമയങ്ങളും ഉള്ള பல்வ...,വ്യക്തമായ ലക്ഷ്യത്തോടെയും സമയക്രമത്തോടെയുമാണ് ...,-0.645948,0.544526,P,0.766826,0.233174
4,ജന്‍ ധന്‍ അക്കൗണ്ടുകള്‍ തുറക്കുന്നതിലും ദരിദ്ര...,"ഈ കാര്യത്തില് അദ്ദേഹം, ജാന് ദന് അക്കൗണ്ട് തുറക...","""ജൻധൻ അക്കൌണ്ടുകൾ തുറക്കൽ, പാവപ്പെട്ടവരുടെ ഇൻഷ...",-0.914935,0.929445,P,0.863466,0.136534


In [43]:
combined['IndicT_NP_prob'] = indictrans[[0]]
combined['IndicT_P_prob'] = indictrans[[1]]
combined['model_pred_IndicT'] = ['NP' if x==0 else 'P' for x in indictrans[2]]
combined['indicT_P'] = apply_softmax(indictrans, 0)
combined['indicT_NP'] = apply_softmax(indictrans, 1)

In [44]:
combined.head()

,Original_Sentence,mBART_Translated,IndicTrans_Translated,mBART_NP_prob,mBART_P_prob,model_pred_mbart,mbart_P,mbart_NP,IndicT_NP_prob,IndicT_P_prob,model_pred_IndicT,indicT_P,indicT_NP
0,കോടിക്കണക്കിന് ജനങ്ങളുടെ മനസില്‍ അഭിലാഷങ്ങള്‍ ...,കോടിക്കണക്കിന് ജനങ്ങളുടെ മനസ്സിലും മനസ്സിലും അ...,കോടിക്കണക്കിന് ജനങ്ങളുടെ മനസ്സിലും ഹൃദയത്തിലും...,-0.535944,0.373010,P,0.712786,0.287214,-1.021923,0.976919,P,0.880675,0.119325
1,രാജ്യത്തിന്റെ വികസയാത്രയില്‍ പിന്നോക്കം വിട്ടു...,ഇന്ന് ബിജ്യപ്പூரில் ഈ പരിപാടി നടത്തുന്നതിന് റെ...,"""ബിജാപ്പൂരിൽ ഇന്ന് ഈ പരിപാടി സംഘടിപ്പിക്കുന്നത...",1.164668,-1.398593,NP,0.071541,0.928459,0.506923,-0.750715,NP,0.221381,0.778619
2,2022 ഓടെ ഈ ലക്ഷ്യം പൂര്‍ത്തീകരിക്കുകയാണ് ലക്ഷ്...,അദ്ധേഹത്തിന് റെ ലക്ഷ്യം 2022-ല് ഈ ജോലി പൂര് ത്...,2022 ഓടെ ഈ ദൌത്യം പൂർത്തിയാക്കുകയാണ് ലക്ഷ്യമെന...,1.247734,-1.467341,NP,0.062090,0.937910,-0.971697,0.945597,P,0.871836,0.128164
3,സമയബന്ധിതമായും കൃത്യമായ ലക്ഷ്യങ്ങളോടുംകൂടി ഗവണ...,കൃത്യമായ ലക്ഷ്യങ്ങളും കൃത്യസമയങ്ങളും ഉള്ള பல்வ...,വ്യക്തമായ ലക്ഷ്യത്തോടെയും സമയക്രമത്തോടെയുമാണ് ...,-0.645948,0.544526,P,0.766826,0.233174,-1.004650,0.952055,P,0.876176,0.123824
4,ജന്‍ ധന്‍ അക്കൗണ്ടുകള്‍ തുറക്കുന്നതിലും ദരിദ്ര...,"ഈ കാര്യത്തില് അദ്ദേഹം, ജാന് ദന് അക്കൗണ്ട് തുറക...","""ജൻധൻ അക്കൌണ്ടുകൾ തുറക്കൽ, പാവപ്പെട്ടവരുടെ ഇൻഷ...",-0.914935,0.929445,P,0.863466,0.136534,-0.951616,0.937276,P,0.868629,0.131371


In [45]:
def select_trasnlation(row):
    if row['indicT_P']>=row['mbart_P']:
        return row['IndicTrans_Translated']
    else:
        return row['mBART_Translated']

combined['selected_translation'] = combined.apply(lambda row : select_trasnlation(row), axis=1) 

In [46]:
combined['mBART_avg'] = combined.iloc[:,3:5].mean(axis=1)

In [47]:
combined.head()

,Original_Sentence,mBART_Translated,IndicTrans_Translated,mBART_NP_prob,mBART_P_prob,model_pred_mbart,mbart_P,mbart_NP,IndicT_NP_prob,IndicT_P_prob,model_pred_IndicT,indicT_P,indicT_NP,selected_translation,mBART_avg
0,കോടിക്കണക്കിന് ജനങ്ങളുടെ മനസില്‍ അഭിലാഷങ്ങള്‍ ...,കോടിക്കണക്കിന് ജനങ്ങളുടെ മനസ്സിലും മനസ്സിലും അ...,കോടിക്കണക്കിന് ജനങ്ങളുടെ മനസ്സിലും ഹൃദയത്തിലും...,-0.535944,0.373010,P,0.712786,0.287214,-1.021923,0.976919,P,0.880675,0.119325,കോടിക്കണക്കിന് ജനങ്ങളുടെ മനസ്സിലും ഹൃദയത്തിലും...,-0.081467
1,രാജ്യത്തിന്റെ വികസയാത്രയില്‍ പിന്നോക്കം വിട്ടു...,ഇന്ന് ബിജ്യപ്പூரில் ഈ പരിപാടി നടത്തുന്നതിന് റെ...,"""ബിജാപ്പൂരിൽ ഇന്ന് ഈ പരിപാടി സംഘടിപ്പിക്കുന്നത...",1.164668,-1.398593,NP,0.071541,0.928459,0.506923,-0.750715,NP,0.221381,0.778619,"""ബിജാപ്പൂരിൽ ഇന്ന് ഈ പരിപാടി സംഘടിപ്പിക്കുന്നത...",-0.116963
2,2022 ഓടെ ഈ ലക്ഷ്യം പൂര്‍ത്തീകരിക്കുകയാണ് ലക്ഷ്...,അദ്ധേഹത്തിന് റെ ലക്ഷ്യം 2022-ല് ഈ ജോലി പൂര് ത്...,2022 ഓടെ ഈ ദൌത്യം പൂർത്തിയാക്കുകയാണ് ലക്ഷ്യമെന...,1.247734,-1.467341,NP,0.062090,0.937910,-0.971697,0.945597,P,0.871836,0.128164,2022 ഓടെ ഈ ദൌത്യം പൂർത്തിയാക്കുകയാണ് ലക്ഷ്യമെന...,-0.109803
3,സമയബന്ധിതമായും കൃത്യമായ ലക്ഷ്യങ്ങളോടുംകൂടി ഗവണ...,കൃത്യമായ ലക്ഷ്യങ്ങളും കൃത്യസമയങ്ങളും ഉള്ള பல்வ...,വ്യക്തമായ ലക്ഷ്യത്തോടെയും സമയക്രമത്തോടെയുമാണ് ...,-0.645948,0.544526,P,0.766826,0.233174,-1.004650,0.952055,P,0.876176,0.123824,വ്യക്തമായ ലക്ഷ്യത്തോടെയും സമയക്രമത്തോടെയുമാണ് ...,-0.050711
4,ജന്‍ ധന്‍ അക്കൗണ്ടുകള്‍ തുറക്കുന്നതിലും ദരിദ്ര...,"ഈ കാര്യത്തില് അദ്ദേഹം, ജാന് ദന് അക്കൗണ്ട് തുറക...","""ജൻധൻ അക്കൌണ്ടുകൾ തുറക്കൽ, പാവപ്പെട്ടവരുടെ ഇൻഷ...",-0.914935,0.929445,P,0.863466,0.136534,-0.951616,0.937276,P,0.868629,0.131371,"""ജൻധൻ അക്കൌണ്ടുകൾ തുറക്കൽ, പാവപ്പെട്ടവരുടെ ഇൻഷ...",0.007255


In [48]:
combined['IT_avg'] = combined.iloc[:,5:7].mean(axis=1)

In [49]:
combined.head()

,Original_Sentence,mBART_Translated,IndicTrans_Translated,mBART_NP_prob,mBART_P_prob,model_pred_mbart,mbart_P,mbart_NP,IndicT_NP_prob,IndicT_P_prob,model_pred_IndicT,indicT_P,indicT_NP,selected_translation,mBART_avg,IT_avg
0,കോടിക്കണക്കിന് ജനങ്ങളുടെ മനസില്‍ അഭിലാഷങ്ങള്‍ ...,കോടിക്കണക്കിന് ജനങ്ങളുടെ മനസ്സിലും മനസ്സിലും അ...,കോടിക്കണക്കിന് ജനങ്ങളുടെ മനസ്സിലും ഹൃദയത്തിലും...,-0.535944,0.373010,P,0.712786,0.287214,-1.021923,0.976919,P,0.880675,0.119325,കോടിക്കണക്കിന് ജനങ്ങളുടെ മനസ്സിലും ഹൃദയത്തിലും...,-0.081467,0.712786
1,രാജ്യത്തിന്റെ വികസയാത്രയില്‍ പിന്നോക്കം വിട്ടു...,ഇന്ന് ബിജ്യപ്പூரில் ഈ പരിപാടി നടത്തുന്നതിന് റെ...,"""ബിജാപ്പൂരിൽ ഇന്ന് ഈ പരിപാടി സംഘടിപ്പിക്കുന്നത...",1.164668,-1.398593,NP,0.071541,0.928459,0.506923,-0.750715,NP,0.221381,0.778619,"""ബിജാപ്പൂരിൽ ഇന്ന് ഈ പരിപാടി സംഘടിപ്പിക്കുന്നത...",-0.116963,0.071541
2,2022 ഓടെ ഈ ലക്ഷ്യം പൂര്‍ത്തീകരിക്കുകയാണ് ലക്ഷ്...,അദ്ധേഹത്തിന് റെ ലക്ഷ്യം 2022-ല് ഈ ജോലി പൂര് ത്...,2022 ഓടെ ഈ ദൌത്യം പൂർത്തിയാക്കുകയാണ് ലക്ഷ്യമെന...,1.247734,-1.467341,NP,0.062090,0.937910,-0.971697,0.945597,P,0.871836,0.128164,2022 ഓടെ ഈ ദൌത്യം പൂർത്തിയാക്കുകയാണ് ലക്ഷ്യമെന...,-0.109803,0.062090
3,സമയബന്ധിതമായും കൃത്യമായ ലക്ഷ്യങ്ങളോടുംകൂടി ഗവണ...,കൃത്യമായ ലക്ഷ്യങ്ങളും കൃത്യസമയങ്ങളും ഉള്ള பல்வ...,വ്യക്തമായ ലക്ഷ്യത്തോടെയും സമയക്രമത്തോടെയുമാണ് ...,-0.645948,0.544526,P,0.766826,0.233174,-1.004650,0.952055,P,0.876176,0.123824,വ്യക്തമായ ലക്ഷ്യത്തോടെയും സമയക്രമത്തോടെയുമാണ് ...,-0.050711,0.766826
4,ജന്‍ ധന്‍ അക്കൗണ്ടുകള്‍ തുറക്കുന്നതിലും ദരിദ്ര...,"ഈ കാര്യത്തില് അദ്ദേഹം, ജാന് ദന് അക്കൗണ്ട് തുറക...","""ജൻധൻ അക്കൌണ്ടുകൾ തുറക്കൽ, പാവപ്പെട്ടവരുടെ ഇൻഷ...",-0.914935,0.929445,P,0.863466,0.136534,-0.951616,0.937276,P,0.868629,0.131371,"""ജൻധൻ അക്കൌണ്ടുകൾ തുറക്കൽ, പാവപ്പെട്ടവരുടെ ഇൻഷ...",0.007255,0.863466


In [50]:
combined.to_csv('../combined-ma.csv', sep='\t', encoding='utf-8')

In [51]:
np.savetxt('../original_sentence-ma.txt', combined['Original_Sentence'], fmt='%s')
np.savetxt('../mBART_translated-ma.txt', combined['mBART_Translated'], fmt='%s')
np.savetxt('../indictrans_translated-ma.txt', combined['IndicTrans_Translated'], fmt='%s')
np.savetxt('../selected_translation-ma.txt', combined['selected_translation'], fmt='%s')

In [52]:
!cp ../original_sentence-ma.txt ../../bleu_test/testing/indicTrans/
!cp ../mBART_translated-ma.txt  ../../bleu_test/testing/indicTrans/
!cp ../indictrans_translated-ma.txt  ../../bleu_test/testing/indicTrans/
!cp ../selected_translation-ma.txt  ../../bleu_test/testing/indicTrans/

In [53]:
!ls -lt ../../bleu_test/testing/indicTrans/*.txt 

-rw-r--r-- 1 root root 286510 Dec  4 16:00 ../../bleu_test/testing/indicTrans/selected_translation-ma.txt
-rw-r--r-- 1 root root 288675 Dec  4 16:00 ../../bleu_test/testing/indicTrans/indictrans_translated-ma.txt
-rw-r--r-- 1 root root 266470 Dec  4 16:00 ../../bleu_test/testing/indicTrans/mBART_translated-ma.txt
-rw-r--r-- 1 root root 314011 Dec  4 16:00 ../../bleu_test/testing/indicTrans/original_sentence-ma.txt
-rw-r--r-- 1 root root 311164 Dec  4 15:54 ../../bleu_test/testing/indicTrans/selected_translation-ta.txt
-rw-r--r-- 1 root root 306457 Dec  4 15:54 ../../bleu_test/testing/indicTrans/indictrans_translated-ta.txt
-rw-r--r-- 1 root root 310154 Dec  4 15:54 ../../bleu_test/testing/indicTrans/mBART_translated-ta.txt
-rw-r--r-- 1 root root 321208 Dec  4 15:54 ../../bleu_test/testing/indicTrans/original_sentence-ta.txt
-rw-r--r-- 1 root root   1000 Dec  4 12:57 ../../bleu_test/testing/indicTrans/mBART_translated.txt
-rw-r--r-- 1 root root      3 Dec  4 12:52 ../../bleu_test/testin